# A guide to fine-tuning Code Llama

**In this guide I show you how to fine-tune Code Llama to become a beast of an SQL developer. For coding tasks, you can generally get much better performance out of Code Llama than Llama 2, especially when you specialise the model on a particular task:**

- I use the [b-mc2/sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context) which is a bunch of text queries and their corresponding SQL queries
- A Lora approach, quantizing the base model to int 8, freezing its weights and only training an adapter
- Much of the code is borrowed from [alpaca-lora](https://github.com/tloen/alpaca-lora), but I refactored it quite a bit for this


### 2. Pip installs


In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb
!pip install scipy

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /var/tmp/pip-req-build-l3o2ep8r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /var/tmp/pip-req-build-l3o2ep8r
  Resolved https://github.com/huggingface/transformers.git to commit 0b192de1f353b0e04dad4813e02e2c672de077be
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /var/tmp/pip-req-build-b_3p_5jc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /var/tmp/pip-req-build-b_3p_5jc
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536

I used an A100 GPU machine with Python 3.10 and cuda 11.8 to run this notebook. It took about an hour to run.

### Loading libraries


In [1]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


(If you have import errors, try restarting your Jupyter kernel)


### Load dataset


In [2]:
from datasets import load_dataset
train_dataset = load_dataset('json', data_files='/home/sam/finetune-llm-for-rag/datasets/sql/MiniLM-L6/samlhuillier-sql-create-context-spider-intersect-train-with-2-examples.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='/home/sam/finetune-llm-for-rag/datasets/sql/MiniLM-L6/samlhuillier-sql-create-context-spider-intersect-validation-with-2-examples.jsonl', split='train')
print(train_dataset)
print(eval_dataset)

Found cached dataset json (/home/sam/.cache/huggingface/datasets/json/default-8e74ab5de84b7560/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Found cached dataset json (/home/sam/.cache/huggingface/datasets/json/default-d78df93a57db14be/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 3961
})
Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 568
})


In [18]:
for x in eval_dataset:
    print(x)
    

{'db_id': 'concert_singer', 'context': 'CREATE TABLE singer (Id VARCHAR)', 'question': 'How many singers do we have?', 'answer': 'SELECT count(*) FROM singer', 'full_prompt': 'You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\nGiven the following examples:\nExample 1:\n### Input:\nHow many actors are there?\n\n### Context:\nCREATE TABLE actor (Id VARCHAR)\n\n### Response:\nSELECT count(*) FROM actor\n\nExample 2:\n### Input:\nHow many employees do we have?\n\n### Context:\nCREATE TABLE Employee (Id VARCHAR)\n\n### Response:\nSELECT count(*) FROM Employee\n\nPlease generate the SQL query that answers the following:\n### Input:\nHow many singers do we have?\n\n### Context:\nCREATE TABLE singer (Id VARCHAR)\n\n### Response:\nSELECT count(*) FROM singer', 'inference_prompt': 'You are a powerful text-to-SQL model. Your job is to 

The above pulls the dataset from the Huggingface Hub and splits 10% of it into an evaluation set to check how well the model is doing through training. If you want to load your own dataset do this:

```
train_dataset = load_dataset('json', data_files='train_set.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='validation_set.jsonl', split='train')
```

And if you want to view any samples in the dataset just do something like:``` ```


In [3]:
print(train_dataset[3])

{'db_id': 'department_management', 'context': 'CREATE TABLE department (budget_in_billions INTEGER)', 'question': 'What are the maximum and minimum budget of the departments?', 'answer': 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department', 'full_prompt': 'You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\nGiven the following examples:\nExample 1:\n### Input:\nList the creation year, name and budget of each department.\n\n### Context:\nCREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)\n\n### Response:\nSELECT creation ,  name ,  budget_in_billions FROM department\n\nExample 2:\n### Input:\nIn which year were most departments established?\n\n### Context:\nCREATE TABLE department (creation VARCHAR)\n\n### Response:\nSELECT creation FROM department GROUP BY creation ORDE

Each entry is made up of a text 'question', the sql table 'context' and the 'answer'.

### Load model
I load code llama from huggingface in int8. Standard for Lora:

In [4]:
base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


torch_dtype=torch.float16 means computations are performed using a float16 representation, even though the values themselves are 8 bit ints.

If you get error "ValueError: Tokenizer class CodeLlamaTokenizer does not exist or is not currently imported." Make sure you have transformers version is 4.33.0.dev0 and accelerate is >=0.20.3.


### 3. Check base model
A very good common practice is to check whether a model can already do the task at hand. Fine-tuning is something you want to try to avoid at all cost:


In [5]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
SELECT * FROM table_name_12 WHERE class > '91.5' AND city_of_license = 'hyannis'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_lic


I get the output:
```
SELECT * FROM table_name_12 WHERE class > 91.5 AND city_of_license = 'hyannis, nebraska'
```
which is clearly wrong if the input is asking for just class!

### 4. Tokenization
Setup some tokenization settings like left padding because it makes [training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa):

In [6]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

Setup the tokenize function to make labels and input_ids the same. This is basically what [self-supervised fine-tuning](https://neptune.ai/blog/self-supervised-learning) is:

In [7]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=700,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

And run convert each data_point into a prompt that I found online that works quite well:

In [8]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = data_point["full_prompt"]
    return tokenize(full_prompt)

Reformat to prompt and tokenize each sample:

In [9]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Loading cached processed dataset at /home/sam/.cache/huggingface/datasets/json/default-8e74ab5de84b7560/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-2499a002c5c14ab5.arrow
Loading cached processed dataset at /home/sam/.cache/huggingface/datasets/json/default-d78df93a57db14be/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-ad98e389bdcd361d.arrow


### 5. Setup Lora

In [10]:
model.train() # put model back into training mode
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

Optional stuff to setup Weights and Biases to view training graphs:

In [11]:
wandb_project = "testing-dataloaders"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project


In [12]:
if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

In [13]:
import numpy as np
from transformers import EvalPrediction

def compute_perplexity(eval_pred: EvalPrediction):
    """
    Compute perplexity for HuggingFace Transformers evaluation.
    
    Parameters:
        eval_pred (EvalPrediction): A namedtuple containing two elements:
            - predictions (np.array): Predicted logits from the model.
            - label_ids (np.array): Ground truth labels.
            
    Returns:
        dict: A dictionary containing the perplexity score.
    """
    print("COMPUTING PERPLEXITY")
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    # Exclude padding tokens (often with id=-100)
    mask = labels != -100
    logits = logits[mask]
    labels = labels[mask]

    # Compute the cross entropy
    cross_entropy = -np.log(logits[np.arange(len(logits)), labels])
    mean_cross_entropy = cross_entropy.mean()

    # Calculate the perplexity
    perplexity = np.exp(mean_cross_entropy)

    return {"perplexity": perplexity}

def dummy_metric(eval_pred):
    """
    A dummy metric function for debugging purposes.
    
    Parameters:
        eval_pred (tuple): A tuple containing predictions and labels. 
                           We won't use them in this function.
            
    Returns:
        dict: A dictionary containing a dummy metric value.
    """
    return {"dummy_metric": 42.0}


### 6. Training arguments
If you run out of GPU memory, change per_device_train_batch_size. The gradient_accumulation_steps variable should ensure this doesn't affect batch dynamics during the training run. All the other variables are standard stuff that I wouldn't recommend messing with:

In [14]:
batch_size = 128
per_device_train_batch_size = 1
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "testing-dataloader"

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=1,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=1,
        save_steps=50,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="none", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
    compute_metrics=compute_perplexity
)

In [15]:
print(model.forward.__doc__)


None


Then we do some pytorch-related optimisation (which just make training faster but don't affect accuracy):

In [16]:
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
    model, type(model)
)
if torch.__version__ >= "2" and sys.platform != "win32":
    print("compiling the model")
    model = torch.compile(model)

compiling the model


In [17]:
trainer.train()

You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1589 in train                    │
│                                                                                                  │
│   1586 │   │   │   finally:                                                                      │
│   1587 │   │   │   │   hf_hub_utils.enable_progress_bars()                                       │
│   1588 │   │   else:                                                                             │
│ ❱ 1589 │   │   │   return inner_training_loop(                                                   │
│   1590 │   │   │   │   args=args,                                                                │
│   1591 │   │   │   │   resume_from_checkpoint=resume_from_checkpoint,                            │
│   1592 │   │   │   │   trial=trial,                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1982 in _inner_training_loop     │
│                                                                                                  │
│   1979 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   1980 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   1981 │   │   │   │   │                                                                         │
│ ❱ 1982 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   1983 │   │   │   │   else:                                                                     │
│   1984 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   1985                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:2326 in _maybe_log_save_evaluate │
│                                                                                                  │
│   2323 │   │   │   │   │   )                                                                     │
│   2324 │   │   │   │   │   metrics.update(dataset_metrics)                                       │
│   2325 │   │   │   else:                                                                         │
│ ❱ 2326 │   │   │   │   metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)                 │
│   2327 │   │   │   self._report_to_hp_search(trial, self.state.global_step, metrics)             │
│   2328 │   │   │                                                                                 │
│   2329 │   │   │   # Run delayed LR scheduler now that metrics are populated                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3064 in evaluate                 │
│                                                                                                  │
│   3061 │   │   start_time = time.time()                                                          │
│   3062 │   │                                               

In [16]:
import sys
print(sys.executable)

/opt/conda/bin/python
